In [2]:
# -------
# IMPORT LIBRAIRIES
# -------
import numpy as np
import pandas as pd
from tqdm import tqdm
import plotly.graph_objects as go
from python_module.pricing_model import BSMModel

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.5f}'.format

In [3]:
S = 100
K = 100
T = 1
r = 0.0
g = 0.0
q = 0.0
sigma = 0.2
n_steps = 252
n_paths = 1000
option_type = 'call'

In [4]:
price_df = BSMModel.compute_montecarlo(S=S, T=T, r=r, g=g, q=q, sigma=sigma, n_steps=n_steps, n_paths=n_paths)
t_df = price_df.copy()
t_df.loc[:, :] = T / n_steps
t_df = t_df.sort_index(ascending=False).cumsum().sort_index()
t_df = t_df - (T / n_steps)

In [5]:
# Compute average delta
greek = price_df.copy()
greek.loc[:, :] = np.nan
for index in tqdm(price_df.index):
    for column in price_df.columns:
        S = price_df.loc[index, column]
        T = t_df.loc[index, column]
        result = BSMModel.compute_option_with_spot(S=S, K=K, T=T, r=r, g=g, q=q, sigma=sigma, option_type=option_type, compute_greeks=True)
        greek.loc[index, column] = result['gamma']
print(greek.iloc[:-1].mean().mean())
print(greek.iloc[:-1].mean(axis=1).mean())

100%|██████████| 253/253 [00:54<00:00,  4.68it/s]

0.020205258721020365
0.02020525872102036


In [6]:
# Compute average slide
bump = -0.3
greek = price_df.copy()
greek.loc[:, :] = np.nan

for index in tqdm(price_df.index):
    for column in price_df.columns:
        
        S = price_df.loc[index, column]
        T = t_df.loc[index, column]
        
        result = BSMModel.compute_option_with_spot(S=S, K=K, T=T, r=r, g=g, q=q, sigma=sigma, option_type=option_type, compute_greeks=True)
        delta_pnl = result['delta'] * S * 0.3
        
        pv_bumped = BSMModel.compute_option_with_spot(S=S*(1+bump), K=K, T=T, r=r, g=g, q=q, sigma=sigma, option_type=option_type, compute_greeks=False)
        option_pnl = pv_bumped - result['price']
        
        greek.loc[index, column] = option_pnl + delta_pnl
        
print(greek.iloc[:-1].mean().mean())
print(greek.iloc[:-1].mean(axis=1).mean())

100%|██████████| 253/253 [01:06<00:00,  3.83it/s]

8.513813160920856
8.513813160920856
